Limpieza para los datos 2.

In [621]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

Funciones

In [622]:
def separar_df(df):
    filtro = df['torneo'].str.contains('Men|ATP', regex=True, case = False)
    masc = df[filtro]
    return masc

def asignar_año(fecha_str):
    # Limpiar y convertir a día y mes (sin año)
    dia_mes = fecha_str.strip('/')  # Elimina '/' final
    dia, mes = map(int, dia_mes.split('/'))
    
    # Asignar 2024 si es antes de 11 de junio, 2025 en caso contrario
    if (mes < 6) or (mes == 6 and dia <= 11):
        return f"{fecha_str}2025"
    else:
        return f"{fecha_str}2024"

def df_jugador(jugador, masc, sup):

    filtro_jugador = (masc['jugadorA'] == jugador) | (masc['jugadorB'] == jugador)
    df_jugador = masc[filtro_jugador].copy()

    df_jugador_A = df_jugador[df_jugador['jugadorA'] == jugador].copy()
    df_jugador_B = df_jugador[df_jugador['jugadorB'] == jugador].copy()

    columnas_A = [col for col in df_jugador.columns if col.endswith('A')]
    columnas_B = [col for col in df_jugador.columns if col.endswith('B')]
    mapeo_columnas = {**{col_a: col_b for col_a, col_b in zip(columnas_A, columnas_B)},
                    **{col_b: col_a for col_a, col_b in zip(columnas_A, columnas_B)}}

    df_jugador_B_invertido = df_jugador_B.rename(columns=mapeo_columnas)

    df_final = pd.concat([df_jugador_A, df_jugador_B_invertido], ignore_index=True)

    df = df_final[df_final['superficie'] == sup]

    df = df[(df['setsA'] != 3) & (df['setsB'] != 3)]

    df = df[df['acesA'].notna() & df['acesB'].notna()]

    if 'fecha' in df.columns:
        df = df.sort_values('fecha', ascending=False)

    return df

def pasarVariablesStringAVariasVariables(masc):
    columnas_a_eliminar = ['juegosGanadosPrimerSaqueA', 'juegosGanadosPrimerSaqueB', 'JuegosGanadosSegundoSaqueA', 'JuegosGanadosSegundoSaqueB', 'BreakPointsSalvadosA', 'BreakPointsSalvadosB', 'PuntosBreakConvertidosA', 'PuntosBreakConvertidosB',
    'PuntosGanadosAlRestoConPrimerSaqueA', 'PuntosGanadosAlRestoConPrimerSaqueB', 'PuntosGanadosAlRestoA', 'PuntosGanadosAlRestoB', 'PuntosGanadosAlRestoConSegundoSaqueA', 'PuntosGanadosAlRestoConSegundoSaqueB', 'PuntosGanadosAlSaqueA', 
    'PuntosGanadosAlSaqueB', 'PuntosGanadosA', 'PuntosGanadosB', 'JuegosGanadosAlSaqueA', 'JuegosGanadosAlSaqueB', 'JuegosGanadosAlRestoA', 'JuegosGanadosAlRestoB', 'JuegosGanadosA', 'JuegosGanadosB']

    # Dividir sin convertir aún
    split_data = masc['PuntosGanadosAlRestoConPrimerSaqueB'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoPrimerSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['juegosGanadosPrimerSaqueA'].str.split('/', expand=True)
    masc['puntosGanadosPrimerSaqueA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosGanadosRestoPrimerSaqueB'] = a
    masc['puntosJugadosPrimerSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['PuntosGanadosAlRestoConPrimerSaqueA'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoPrimerSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['juegosGanadosPrimerSaqueB'].str.split('/', expand=True)
    masc['puntosGanadosPrimerSaqueB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosGanadosRestoPrimerSaqueA'] = a
    masc['puntosJugadosPrimerSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['PuntosGanadosAlRestoConSegundoSaqueB'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoSegundoSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['JuegosGanadosSegundoSaqueA'].str.split('/', expand=True)
    masc['puntosGanadosSegundoSaqueA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosGanadosRestoSegundoSaqueB'] = a
    masc['puntosJugadosSegundoSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['PuntosGanadosAlRestoConSegundoSaqueA'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoSegundoSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['JuegosGanadosSegundoSaqueB'].str.split('/', expand=True)
    masc['puntosGanadosSegundoSaqueB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosGanadosRestoSegundoSaqueA'] = a
    masc['puntosJugadosSegundoSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['BreakPointsSalvadosA'].str.split('/', expand=True)
    masc['breakPointsSalvadosA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['breakPointsTotalesEnContraA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['PuntosBreakConvertidosB'].str.split('/', expand=True)
    masc['breakPointsGanadosB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['breakPointsTotalesB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['BreakPointsSalvadosB'].str.split('/', expand=True)
    masc['breakPointsSalvadosB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['breakPointsTotalesEnContraB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['PuntosBreakConvertidosA'].str.split('/', expand=True)
    masc['breakPointsGanadosA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['breakPointsTotalesA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['PuntosGanadosAlRestoA'].str.split('/', expand=True)
    masc['puntosGanadosRestoA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['PuntosGanadosAlSaqueB'].str.split('/', expand=True)
    masc['puntosGanadosSaqueB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosJugadosSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['PuntosGanadosAlRestoB'].str.split('/', expand=True)
    masc['puntosGanadosRestoB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosRestoB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['PuntosGanadosAlSaqueA'].str.split('/', expand=True)
    masc['puntosGanadosSaqueA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosJugadosSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')


    split_data = masc['PuntosGanadosB'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['puntosJugadosB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['PuntosGanadosA'].str.split('/', expand=True)
    masc['puntosGanadosA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['puntosGanadosB'] = a
    masc['puntosJugados'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')


    # Dividir sin convertir aún
    split_data = masc['JuegosGanadosAlSaqueA'].str.split('/', expand=True)
    masc['juegosGanadosSaqueA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['juegosJugadosSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['JuegosGanadosAlRestoB'].str.split('/', expand=True)
    masc['juegosGanadosRestoB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['juegosJugadosSaqueA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['JuegosGanadosAlSaqueB'].str.split('/', expand=True)
    masc['juegosGanadosSaqueB'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['juegosJugadosSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['JuegosGanadosAlRestoA'].str.split('/', expand=True)
    masc['juegosGanadosRestoA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['juegosJugadosSaqueB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    split_data = masc['JuegosGanadosB'].str.split('/', expand=True)
    a = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    #masc['juegosJugadosB'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    # Dividir sin convertir aún
    split_data = masc['JuegosGanadosA'].str.split('/', expand=True)
    masc['juegosGanadosA'] = pd.to_numeric(split_data[0], errors='coerce').astype('Int64')
    masc['juegosGanadosB'] = a
    masc['juegosJugadosA'] = pd.to_numeric(split_data[1], errors='coerce').astype('Int64')

    masc = masc.drop(columns = columnas_a_eliminar)

    return masc

Lectura

In [644]:
masc = pd.read_csv(".\\data\\data2\\masc.csv", low_memory=False)

masc1 = pd.read_csv(".\\data\\data2\\montecarlo-palermo_masc_2025-06-13.csv", low_memory=False)
masc2 = pd.read_csv(".\\data\\data1\\lyon_perugia_masc_2025-06-11.csv", low_memory=False)
masc3 = pd.read_csv(".\\data\\data1\\partidos_tenis_2025-06-11.csv", low_memory=False)
masc4 = pd.read_csv(".\\data\\data1\\santafe_masc_2025-06-11.csv", low_memory=False)
masc5 = pd.read_csv(".\\data\\data1\\villeneuveloubet-varnamo_masc_2025-06-11.csv", low_memory=False)

print(len(masc))

30484


In [645]:
display(masc.head())
display(masc1.head())

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,dobleFaltasB,porcentajePrimerSaqueA,porcentajePrimerSaqueB,porcentajePuntosGanadosConPrimerSaqueA,porcentajePuntosGanadosConPrimerSaqueB,puntosGanadosConPrimerSaqueA,puntosGanadosConPrimerSaqueB,porcentajePuntosGanadosConSegundoSaqueA,porcentajePuntosGanadosConSegundoSaqueB,puntosGanadosConSegundoSaqueA,puntosGanadosConSegundoSaqueB,porcentajeBreakPointsSalvadosA,porcentajeBreakPointsSalvadosB,breakPointsSalvadosA,breakPointsSalvadosB,porcentajeBreakPointsA,porcentajeBreakPointsB,breakPointsA,breakPointsB,velocidadMediaPrimerSaqueA,velocidadMediaPrimerSaqueB,velocidadMediaSegundoSaqueA,velocidadMediaSegundoSaqueB,porcentajePuntosGanadosAlRestoConPrimerSaqueA,porcentajePuntosGanadosAlRestoConPrimerSaqueB,puntosGanadosAlRestoConPrimerSaqueA,puntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoConSegundoSaqueA,porcentajePuntosGanadosAlRestoConSegundoSaqueB,puntosGanadosAlRestoConSegundoSaqueA,puntosGanadosAlRestoConSegundoSaqueB,winnersA,winnersB,erroresNoForzadosA,erroresNoForzadosB,porcentajePuntosGanadosRedA,porcentajePuntosGanadosRedB,puntosGanadosRedA,puntosGanadosRedB,porcentajePuntosGanadosSaqueA,porcentajePuntosGanadosSaqueB,puntosGanadosSaqueA,puntosGanadosSaqueB,porcentajePuntosGanadosRestoA,porcentajePuntosGanadosRestoB,puntosGanadosRestoA,puntosGanadosRestoB,porcentajePuntosGanadosA,porcentajePuntosGanadosB,puntosGanadosA,puntosGanadosB,puntosGanadosUltimas10BolasA,puntosGanadosUltimas10BolasB,puntosDePartidoSalvadosA,puntosDePartidoSalvadosB,porcentajeJuegosGanadosSaqueA,porcentajeJuegosGanadosSaqueB,juegosGanadosSaqueA,juegosGanadosSaqueB,porcentajeJuegosGanadosRestoA,porcentajeJuegosGanadosRestoB,juegosGanadosRestoA,juegosGanadosRestoB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,juegosGanadosA,juegosGanadosB,cuotaA,cuotaB,cuotaA_2,cuotaB_2,dobleA,dobleB,porcentajeJuegosGanadosPrimerSaqueA,juegosGanadosPrimerSaqueA,porcentajeJuegosGanadosPrimerSaqueB,juegosGanadosPrimerSaqueB,porcentajeJuegosGanadosSegundoSaqueA,JuegosGanadosSegundoSaqueA,porcentajeJuegosGanadosSegundoSaqueB,JuegosGanadosSegundoSaqueB,BreakPointsSalvadosA,BreakPointsSalvadosB,PuntosGanadosAlRestoConPrimerSaqueA,PuntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoA,PuntosGanadosAlRestoA,porcentajePuntosGanadosAlRestoB,PuntosGanadosAlRestoB,PuntosGanadosAlRestoConSegundoSaqueA,PuntosGanadosAlRestoConSegundoSaqueB,porcentajePuntosBreakConvertidosA,PuntosBreakConvertidosA,porcentajePuntosBreakConvertidosB,PuntosBreakConvertidosB,porcentajePuntosGanadosAlSaqueA,PuntosGanadosAlSaqueA,porcentajePuntosGanadosAlSaqueB,PuntosGanadosAlSaqueB,PuntosGanadosA,PuntosGanadosB,PuntosGanadosUltimas10BolasA,PuntosGanadosUltimas10BolasB,BolasDePartidoSalvadasA,BolasDePartidoSalvadasB,porcentajeJuegosGanadosAlSaqueA,JuegosGanadosAlSaqueA,porcentajeJuegosGanadosAlSaqueB,JuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,ano,PuntosGanadosRedA,PuntosGanadosRedB
0,acapulco,Machac T.,Davidovich Fokina A.,2,0,02.03. 04:10,no,11.0,4.0,NaN,NaN,64%,59%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100%,33%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38%,5%,NaN,NaN,38%,45%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58%,42%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60%,40%,NaN,NaN,1.67,2.20,1.67,2.2,2.0,2.0,95%,(37/39),62%,(26/42),55%,(12/22),62%,(18/29),(1/1),(1/3),(16/42),(2/39),38%,(27/71),20%,(12/61),(11/29),(10/22),67%,(2/3),0%,(0/1),80%,(49/61),62%,(44/71),(76/132),(56/132),8.0,2.0,0.0,1.0,100%,(10/10),80%,(8/10),20%,(2/10),0%,(0/10),(12/20),(8/20),2025.0,NaN,NaN
1,acapulco,Shapovalov D.,Davidovich Fokina A.,0,2,01.03. 04:40,no,2.0,0.0,NaN,NaN,54%,70%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60%,43%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35%,37%,NaN,NaN,50%,56%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47%,53%,Na

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,ano,acesA,acesB,dobleFaltasA,dobleFaltasB,porcentajePrimerSaqueA,porcentajePrimerSaqueB,porcentajePuntosGanadosConPrimerSaqueA,porcentajePuntosGanadosConPrimerSaqueB,puntosGanadosConPrimerSaqueA,puntosGanadosConPrimerSaqueB,porcentajePuntosGanadosConSegundoSaqueA,porcentajePuntosGanadosConSegundoSaqueB,puntosGanadosConSegundoSaqueA,puntosGanadosConSegundoSaqueB,porcentajeBreakPointsSalvadosA,porcentajeBreakPointsSalvadosB,breakPointsSalvadosA,breakPointsSalvadosB,porcentajeBreakPointsA,porcentajeBreakPointsB,breakPointsA,breakPointsB,velocidadMediaPrimerSaqueA,velocidadMediaPrimerSaqueB,velocidadMediaSegundoSaqueA,velocidadMediaSegundoSaqueB,porcentajePuntosGanadosAlRestoConPrimerSaqueA,porcentajePuntosGanadosAlRestoConPrimerSaqueB,puntosGanadosAlRestoConPrimerSaqueA,puntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoConSegundoSaqueA,porcentajePuntosGanadosAlRestoConSegundoSaqueB,puntosGanadosAlRestoConSegundoSaqueA,puntosGanadosAlRestoConSegundoSaqueB,winnersA,winnersB,erroresNoForzadosA,erroresNoForzadosB,porcentajePuntosGanadosRedA,porcentajePuntosGanadosRedB,puntosGanadosRedA,puntosGanadosRedB,porcentajePuntosGanadosSaqueA,porcentajePuntosGanadosSaqueB,puntosGanadosSaqueA,puntosGanadosSaqueB,porcentajePuntosGanadosRestoA,porcentajePuntosGanadosRestoB,puntosGanadosRestoA,puntosGanadosRestoB,porcentajePuntosGanadosA,porcentajePuntosGanadosB,puntosGanadosA,puntosGanadosB,puntosGanadosUltimas10BolasA,puntosGanadosUltimas10BolasB,puntosDePartidoSalvadosA,puntosDePartidoSalvadosB,porcentajeJuegosGanadosSaqueA,porcentajeJuegosGanadosSaqueB,juegosGanadosSaqueA,juegosGanadosSaqueB,porcentajeJuegosGanadosRestoA,porcentajeJuegosGanadosRestoB,juegosGanadosRestoA,juegosGanadosRestoB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,juegosGanadosA,juegosGanadosB,cuotaA,cuotaB,cuotaA_2,cuotaB_2,dobleA,dobleB,porcentajeJuegosGanadosPrimerSaqueA,juegosGanadosPrimerSaqueA,porcentajeJuegosGanadosPrimerSaqueB,juegosGanadosPrimerSaqueB,porcentajeJuegosGanadosSegundoSaqueA,JuegosGanadosSegundoSaqueA,porcentajeJuegosGanadosSegundoSaqueB,JuegosGanadosSegundoSaqueB,BreakPointsSalvadosA,BreakPointsSalvadosB,PuntosGanadosAlRestoConPrimerSaqueA,PuntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoA,PuntosGanadosAlRestoA,porcentajePuntosGanadosAlRestoB,PuntosGanadosAlRestoB,PuntosGanadosAlRestoConSegundoSaqueA,PuntosGanadosAlRestoConSegundoSaqueB,porcentajePuntosBreakConvertidosA,PuntosBreakConvertidosA,porcentajePuntosBreakConvertidosB,PuntosBreakConvertidosB,PuntosGanadosRedA,PuntosGanadosRedB,porcentajePuntosGanadosAlSaqueA,PuntosGanadosAlSaqueA,porcentajePuntosGanadosAlSaqueB,PuntosGanadosAlSaqueB,PuntosGanadosA,PuntosGanadosB,PuntosGanadosUltimas10BolasA,PuntosGanadosUltimas10BolasB,BolasDePartidoSalvadasA,BolasDePartidoSalvadasB,porcentajeJuegosGanadosAlSaqueA,JuegosGanadosAlSaqueA,porcentajeJuegosGanadosAlSaqueB,JuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB
0,monte-carlo,Musetti L.,Alcaraz C.,1,2,13.04. 12:10,no,2025,3,3,NaN,NaN,63%,72%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25%,78%,NaN,NaN,NaN,NaN,NaN,NaN,181 km/h,186 km/h,151 km/h,163 km/h,30%,51%,NaN,NaN,50%,48%,NaN,NaN,10.0,30.0,29.0,32.0,50%,76%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42%,58%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32%,68%,NaN,NaN,4.50,1.20,4.50,1.20,1.0,3.0,49%,(21/43),70%,(39/56),52%,(13/25),50%,(11/22),(2/8),(7/9),(17/56),(22/43),36%,(28/78),50%,(34/68),(11/22),(12/25),22%,(2/9),75%,(6/8),(2/4),(13/17),50%,(34/68),64%,(50/78),(62/146),(84/146),2.0,8.0,0.0,0.0,45%,(5/11),82%,(9/11),18%,(2/11),55%,(6/11),(7/22),(15/22)
1,monte-carlo,Musetti L.,De Minaur A.,2,1,12.04. 15:40,no,2025,2,1,NaN,NaN,67%,52%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42%,44%,NaN,NaN,NaN,NaN,NaN,NaN,188 km/h,203 km/h,141 km/h,158 km/h,27%,40%,NaN,NaN,60%,61%,NaN,NaN,20.0,23.0,43.0,45.0,82%,78%,NaN,NaN,NaN,Na

In [624]:
masc = masc[(masc['setsA'] != 'Isroilov S.')]
masc = masc[(masc['setsA'] != 'Hanzlik C.')]
masc = masc[(masc['setsA'] != '6')]
masc = masc[(masc['setsB'] != '5')]
masc = masc[(masc['setsB'] != '7')]

jugadores_A = masc['jugadorA'].unique().tolist()
jugadores_B = masc['jugadorB'].unique().tolist()
todos_jugadores = list(set(jugadores_A + jugadores_B))
masc = masc[~masc['setsA'].isin(todos_jugadores)]

masc['setsA'] = masc['setsA'].astype(int)
masc['setsB'] = masc['setsB'].astype(int)

filtroGrandSlam = (masc['torneo'] == 'australian-open') | (masc['torneo'] == 'us-open') | (masc['torneo'] == 'wimbledon') | (masc['torneo'] == 'french-open')
filtro3sets = (masc['setsA'] == 3) | (masc['setsB'] == 3)
filtro2sets = (masc['setsA'] == 2) | (masc['setsB'] == 2)
filtroGrandSlamGanaA = masc['setsA'] == 3
filtroGrandSlamGanaB = masc['setsB'] == 3
filtroNoGrandSlamGanaA = masc['setsA'] == 2
filtroNoGrandSlamGanaB = masc['setsB'] == 2

# Condición 1: es Grand Slam y no se llegó a 3 sets (partido incompleto/retirado)
cond1 = filtroGrandSlam & (~filtro3sets)
masc.loc[cond1, 'retirado'] = 'si'

# Condición 2: no es Grand Slam y no se llegó a 2 sets (partido incompleto/retirado)
cond2 = (~filtroGrandSlam) & (~filtro2sets)
masc.loc[cond2, 'retirado'] = 'si'

#Creo la variable superficie
torneos_superficies = [
    ('acapulco', 'Hard'), ('adelaide', 'Hard'), ('adelaide-2', 'Hard'), ('almaty', 'Clay'), ('almaty-2', 'Clay'), ('amersfoort', 'Clay'), ('amsterdam', 'Clay'), ('antalya', 'Grass'), 
    ('antalya-2', 'Grass'), ('antalya-3', 'Grass'), ('antalya-4', 'Grass'), ('antwerp', 'Hard'), ('antwerp-2', 'Hard'), ('asian-games', 'Hard'), ('astana', 'Hard'), ('astana-2', 'Hard'), 
    ('athens', 'Clay'), ('atlanta', 'Hard'), ('atp-cup', 'Hard'), ('auckland', 'Hard'), ('australian-open', 'Hard'), ('banja-luka', 'Clay'), ('bangkok', 'Hard'), ('bangkok-2', 'Hard'), 
    ('bangkok-3', 'Hard'), ('bangkok-4', 'Hard'), ('barcelona', 'Clay'), ('barletta', 'Clay'), ('basel', 'Hard'), ('bastad', 'Clay'), ('beijing', 'Hard'), ('belgrade', 'Clay'), 
    ('belgrade-2', 'Clay'), ('bergamo', 'Hard'), ('berlin', 'Grass'), ('bermuda', 'Hard'), ('biella', 'Hard'), ('biella-2', 'Hard'), ('biella-3', 'Hard'), ('biella-4', 'Hard'), 
    ('biella-5', 'Hard'), ('biella-6', 'Hard'), ('biella-7', 'Hard'), ('biel', 'Hard'), ('birmingham', 'Grass'), ('blois', 'Clay'), ('bogota', 'Clay'), ('bogota-2', 'Clay'), 
    ('bogota-3', 'Clay'), ('bolzano', 'Hard'), ('bordeaux', 'Clay'), ('boston', 'Hard'), ('brasilia', 'Clay'), ('brighton', 'Hard'), ('brisbane', 'Hard'), ('brisbane-2', 'Hard'), 
    ('bucharest', 'Clay'), ('budapest', 'Clay'), ('buenos-aires', 'Clay'), ('buenos-aires-2', 'Clay'), ('buenos-aires-3', 'Clay'), ('buzios', 'Clay'), ('cagliari', 'Clay'), ('calgary', 'Hard'), 
    ('campinas', 'Clay'), ('canberra', 'Hard'), ('canberra-2', 'Hard'), ('cap-cana', 'Hard'), ('casablanca', 'Clay'), ('casablanca-2', 'Clay'), ('cassis', 'Hard'), ('cary', 'Hard'), 
    ('cary-2', 'Hard'), ('champaign', 'Hard'), ('chengdu', 'Hard'), ('chennai', 'Hard'), ('chicago', 'Hard'), ('chitre', 'Hard'), ('chongqing', 'Hard'), ('cincinnati', 'Hard'), 
    ('cleveland', 'Hard'), ('cologne', 'Hard'), ('cologne-2', 'Hard'), ('como', 'Clay'), ('concepcion', 'Clay'), ('concepcion-2', 'Clay'), ('constanta', 'Clay'), ('copenhagen', 'Hard'), 
    ('cordoba', 'Clay'), ('cordoba-2', 'Clay'), ('corrientes', 'Clay'), ('costa-do-sauipe', 'Clay'), ('courmayeur', 'Hard'), ('curitiba', 'Clay'), ('dallas', 'Hard'), ('delray-beach', 'Hard'), 
    ('doha', 'Hard'), ('drummondville', 'Hard'), ('dubai', 'Hard'), ('dusseldorf', 'Clay'), ('eckental', 'Carpet'), ('eastbourne', 'Grass'), ('essen', 'Hard'), ('estoril', 'Clay'), 
    ('fergana', 'Hard'), ('florence', 'Hard'), ('floridablanca', 'Clay'), ('forli', 'Clay'), ('forli-2', 'Clay'), ('forli-3', 'Clay'), ('forli-4', 'Clay'), ('forli-5', 'Clay'), 
    ('forli-6', 'Clay'), ('francavilla', 'Clay'), ('french-open', 'Clay'), ('furth', 'Clay'), ('gijon', 'Hard'), ('gimcheon', 'Hard'), ('glasgow', 'Hard'), ('granby', 'Hard'), 
    ('gran-canaria', 'Hard'), ('gran-canaria-2', 'Hard'), ('grodzisk-mazowiecki', 'Hard'), ('gstaad', 'Clay'), ('guadalajara', 'Hard'), ('guangzhou', 'Hard'), ('guangzhou-2', 'Hard'), 
    ('guayaquil', 'Clay'), ('guimaraes', 'Hard'), ('gwangju', 'Hard'), ('halle', 'Grass'), ('hamilton', 'Hard'), ('hamburg', 'Clay'), ('hangzhou', 'Hard'), ('happy-valley', 'Hard'), 
    ('helsinki', 'Hard'), ('heilbronn', 'Hard'), ('heilbronn-2', 'Hard'), ('hertogenbosch', 'Grass'), ('ho-chi-minh-city', 'Hard'), ('hong-kong', 'Hard'), ('houston', 'Clay'), ('ibague', 'Clay'), 
    ('indian-wells', 'Hard'), ('indian-wells-2', 'Hard'), ('irving', 'Hard'), ('islamabad', 'Hard'), ('istanbul', 'Clay'), ('istanbul-2', 'Clay'), ('itajai', 'Clay'), ('jakarta', 'Hard'), 
    ('jerusalem', 'Hard'), ('jinan', 'Hard'), ('karlsruhe', 'Clay'), ('kazan', 'Hard'), ('kazan-2', 'Hard'), ('kenitra', 'Clay'), ('kigali', 'Hard'), ('kigali-2', 'Hard'), ('kitzbuhel', 'Clay'), ('koblenz', 'Hard'), 
    ('kolding', 'Hard'), ('kolkata', 'Hard'), ('kosice', 'Clay'), ('kyoto', 'Hard'), ('launceston', 'Hard'), ('lille', 'Hard'), ('lima', 'Clay'), ('lima-2', 'Clay'), ('lisbon', 'Hard'), ('little-rock', 'Hard'), 
    ('ljubljana', 'Hard'), ('london', 'Grass'), ('los-cabos', 'Hard'), ('luedenscheid', 'Clay'), ('lugano', 'Clay'), ('lyon', 'Hard'), ('lyon-2', 'Hard'), ('madrid', 'Clay'), ('madrid-2', 'Clay'), 
    ('maia', 'Clay'), ('maia-2', 'Clay'), ('mallorca', 'Grass'), ('manama', 'Hard'), ('manchester', 'Grass'), ('manerbio', 'Clay'), ('marrakech', 'Clay'), ('marseille', 'Hard'), 
    ('matsuyama', 'Hard'), ('mauthausen', 'Clay'), ('medellin', 'Clay'), ('melbourne', 'Hard'), ('menorca', 'Hard'), ('mersin', 'Clay'), ('merida', 'Hard'), ('metz', 'Hard'), 
    ('mexico-city', 'Clay'), ('milan', 'Hard'), ('minsk', 'Hard'), ('modena', 'Clay'), ('monte-carlo', 'Clay'), ('monterrey', 'Hard'), ('montpellier', 'Hard'), ('montreal', 'Hard'), 
    ('moscow', 'Clay'), ('mouilleron-le-captif', 'Hard'), ('munich', 'Clay'), ('murcia', 'Clay'), ('napa', 'Hard'), ('napoli', 'Clay'), ('napoli-2', 'Clay'), ('new-york', 'Hard'), 
    ('newport', 'Grass'), ('newport-beach', 'Hard'), ('nice', 'Clay'), ('ningbo', 'Hard'), ('nonthaburi', 'Hard'), ('nonthaburi-2', 'Hard'), ('nonthaburi-3', 'Hard'), ('nonthaburi-4', 'Hard'), 
    ('noumea', 'Hard'), ('nur-sultan', 'Hard'), ('nur-sultan-2', 'Hard'), ('nur-sultan-3', 'Hard'), ('nur-sultan-4', 'Hard'), ('oeiras', 'Clay'), ('oeiras-2', 'Clay'), ('oeiras-3', 'Clay'), 
    ('oeiras-4', 'Clay'), ('oeiras-5', 'Clay'), ('oeiras-6', 'Clay'), ('olbia', 'Hard'), ('olympic-games', 'Hard'), ('oporto', 'Hard'), ('orleans', 'Hard'), ('orlando', 'Hard'), 
    ('orlando-2', 'Hard'), ('osaka', 'Hard'), ('ostrava', 'Hard'), ('padova', 'Clay'), ('palermo', 'Clay'), ('pau', 'Hard'), ('pereira', 'Clay'), ('perugia', 'Clay'), ('phoenix', 'Hard'), ('piracicaba', 'Clay'), 
    ('playford', 'Hard'), ('playford-2', 'Hard'), ('poertschach', 'Clay'), ('poznan', 'Clay'), ('prague', 'Clay'), ('prague-3', 'Clay'), ('prostejov', 'Clay'), ('puebla', 'Hard'), ('pune', 'Hard'), ('punta-del-este', 'Clay'), 
    ('quito', 'Clay'), ('rabat', 'Clay'), ('raanana', 'Hard'), ('recanati', 'Clay'), ('reggio-emilia', 'Clay'), ('rennes', 'Hard'), ('rhodes', 'Hard'), ('rio-de-janeiro', 'Clay'), 
    ('rio-de-janeiro-2', 'Clay'), ('rimini', 'Clay'), ('rio-quente', 'Hard'), ('roanne', 'Hard'), ('roseto-degli-abruzzi', 'Clay'), ('roseto-degli-abruzzi-2', 'Clay'), ('rotterdam', 'Hard'), 
    ('rovereto', 'Hard'), ('saint-brieuc', 'Hard'), ('salinas', 'Hard'), ('salinas-2', 'Hard'), ('salzburg', 'Clay'), ('san-diego', 'Hard'), ('san-juan', 'Hard'), ('san-luis-potosi', 'Hard'), 
    ('san-marino', 'Clay'), ('san-miguel-de-tucuman', 'Clay'), ('san-remo', 'Clay'), ('san-sebastian', 'Clay'), ('santos', 'Clay'), ('santos-open', 'Clay'), ('sao-paulo', 'Clay'), 
    ('sao-paulo-2', 'Clay'), ('sao-paulo-3', 'Clay'), ('sao-paulo-4', 'Clay'), ('sao-paulo-5', 'Clay'), ('sarajevo', 'Hard'), ('sarasota', 'Clay'), ('santiago', 'Clay'), ('santiago-2', 'Clay'), 
    ('santiago-3', 'Clay'), ('santo-domingo', 'Hard'), ('santos', 'Clay'), ('sardinia', 'Clay'), ('savannah', 'Hard'), ('schenectady', 'Hard'), ('scheveningen', 'Clay'), ('scottsdale', 'Hard'), 
    ('segovia', 'Hard'), ('seoul', 'Hard'), ('shenzhen', 'Hard'), ('shenzhen-2', 'Hard'), ('shenzhen-3', 'Hard'), ('shymkent', 'Hard'), ('shymkent-2', 'Hard'), ('singapore', 'Hard'), 
    ('skopje', 'Clay'), ('sofia', 'Hard'), ('sopot', 'Clay'), ('split', 'Clay'), ('split-2', 'Clay'), ('st-brieuc', 'Hard'), ('stockholm', 'Hard'), ('stockton', 'Hard'), ('surbiton', 'Grass'), 
    ('sydney', 'Hard'), ('sydney-2', 'Hard'), ('szczecin', 'Clay'), ('szekesfehervar', 'Clay'), ('taipei', 'Hard'), ('taipei-2', 'Hard'), ('tallahassee', 'Hard'), ('tampere', 'Hard'), 
    ('tanger', 'Clay'), ('tarragona', 'Clay'), ('tashkent', 'Hard'), ('tel-aviv', 'Hard'), ('tenerife', 'Hard'), ('tenerife-2', 'Hard'), ('tenerife-3', 'Hard'), ('tianjin', 'Hard'), 
    ('tiburon', 'Hard'), ('timisoara', 'Clay'), ('tokyo', 'Hard'), ('tokyo-2', 'Hard'), ('toulouse', 'Hard'),('sassuolo', 'Clay'), ('royan', 'Clay'), ('poznan', 'Clay'), ('nottingham', 'Grass'), 
    ('santa-cruz', 'Hard'), ('ilkley', 'Grass'), ('bratislava', 'Clay'), ('santa-fe', 'Clay'), ('perugia', 'Clay'), ('stuttgart', 'Grass'), ('hertogenbosch', 'Grass'), ('tyler', 'Hard'), 
    ('prostejov', 'Clay'), ('chisinau', 'Clay'), ('vicenza', 'Clay'), ('tbilisi', 'Hard'), ('geneva', 'Clay'), ('turin-2', 'Hard'), ('rome', 'Clay'), ('tunis', 'Clay'), 
    ('zagreb', 'Hard'), ('wuxi', 'Hard'), ('francavilla', 'Clay'), ('ostrava', 'Hard'), ('porto-alegre', 'Clay'), ('aix-en-provence', 'Clay'), ('gwangju', 'Hard'), ('abidjan-2', 'Hard'), 
    ('savannah', 'Hard'), ('abidjan', 'Hard'), ('busan', 'Hard'), ('tallahassee', 'Hard'), ('monza', 'Clay'), ('mexico-city', 'Hard'), ('houston', 'Clay'), ('morelos', 'Hard'), 
    ('menorca', 'Hard'), ('girona', 'Clay'), ('naples', 'Clay'), ('merida', 'Hard'), ('zadar', 'Clay'), ('asuncion', 'Clay'), ('murcia', 'Clay'), ('phoenix', 'Hard'), 
    ('hersonissos-2', 'Hard'), ('cherbourg', 'Hard'), ('hersonissos', 'Hard'), ('thionville', 'Hard'), ('lugano', 'Clay'), ('san-diego', 'Hard'), ('brazzaville', 'Hard'), ('pune', 'Hard'), 
    ('new-delhi', 'Hard'), ('tenerife-2', 'Hard'), ('manama', 'Hard'), ('rosario', 'Clay'), ('tenerife', 'Hard'), ('quimper', 'Hard'), ('oeiras-5', 'Clay'), ('oeiras-2', 'Clay'), 
    ('nonthaburi-3', 'Hard'), ('nottingham-3', 'Grass'), ('nonthaburi-2', 'Hard'), ('hong-kong', 'Hard'), ('united-cup', 'Hard'), ('noumea', 'Hard'), ('manzanillo', 'Hard'), ('yokkaichi', 'Hard'), 
    ('temuco', 'Clay'), ('puerto-vallarta', 'Hard'), ('yokohama', 'Hard'), ('montevideo', 'Clay'), ('kobe', 'Hard'), ('matsuyama', 'Hard'), ('knoxville', 'Hard'), ('guayaquil', 'Clay'), 
    ('charlottesville', 'Hard'), ('bratislava-2', 'Clay'), ('playford-2', 'Hard'), ('sioux-falls', 'Hard'), ('brest', 'Hard'), ('olbia', 'Hard'), ('shenzhen-2', 'Hard'), ('villa-maria', 'Clay'), 
    ('hangzhou', 'Hard'), ('mouilleron-le-captif', 'Hard'), ('alicante', 'Clay'), ('braga', 'Clay'), ('charleston', 'Clay'), ('antofagasta', 'Clay'), ('nonthaburi-4', 'Hard'), ('cali', 'Clay'), 
    ('laver-cup', 'Hard'), ('sibiu', 'Clay'), ('columbus', 'Hard'), ('las-vegas', 'Hard'), ('dobrich-2', 'Clay'), ('genova', 'Clay'), ('tulln', 'Clay'), ('seville', 'Clay'), 
    ('porto-2', 'Hard'), ('winston-salem', 'Hard'), ('dobrich', 'Clay'), ('santo-domingo', 'Hard'), ('grodzisk-mazowiecki', 'Hard'), ('todi', 'Clay'), ('lincoln', 'Hard'), ('bonn', 'Hard'), 
    ('cordenons', 'Clay'), ('liberec', 'Clay'), ('lexington', 'Hard'), ('luedenscheid', 'Clay'), ('verona', 'Clay'), ('segovia', 'Hard'), ('zug', 'Hard'), ('granby', 'Hard'), 
    ('pozoblanco', 'Hard'), ('salzburg', 'Clay'), ('iasi', 'Clay'), ('trieste', 'Clay'),  ('winnipeg', 'Hard'), ('braunschweig', 'Clay'), ('brasov', 'Clay'), ('troyes', 'Clay'), 
    ('bloomfield-hills', 'Hard'), ('ibague', 'Clay'), ('santa-cruz-2', 'Hard'), ('surbiton', 'Grass'), ('kachreti', 'Hard'), ('augsburg', 'Hard'), ('oeiras-4', 'Clay'), ('shenzhen-3', 'Hard'), 
    ('split', 'Clay'), ('florianopolis', 'Clay'), ('sao-leopoldo', 'Clay'), ('szekesfehervar', 'Clay'), ('los-cabos', 'Hard'), ('burnie-2', 'Hard'), ('burnie', 'Hard'), ('ottignies-louvain-la-neuve', 'Hard'), 
    ('maspalomas', 'Hard'), ('danderyd', 'Hard'), ('ortisei', 'Hard'), ('malaga', 'Hard'), ('zhuhai', 'Hard'), ('prague-3', 'Clay'), ('stanford', 'Hard'), ('meerbusch', 'Clay'), 
    ('salinas', 'Hard'), ('san-benedetto', 'Clay'), ('palmas-del-mar', 'Hard'), ('troisdorf', 'Hard'), ('roseto-degli-abruzzi', 'Clay'), ('sanremo', 'Clay'), ('las-franquesas-del-valles', 'Clay'), 
    ('vina-del-mar', 'Clay'), ('waco', 'Hard'), ('rome-us', 'Hard'), ('vilnius', 'Hard'), ('tigre-2', 'Clay'), ('tigre', 'Clay'), ('andria', 'Hard'), ('ambato', 'Clay'), 
    ('gijon', 'Hard'), ('parma-2', 'Clay'), ('tel-aviv', 'Hard'), ('columbus-2', 'Hard'), ('vancouver', 'Hard'), ('indianapolis', 'Hard'), ('orlando', 'Hard'), ('forli-6', 'Clay'), 
    ('shymkent-2', 'Hard'), ('shymkent', 'Hard'), ('aguascalientes', 'Clay'), ('marbella', 'Clay'), ('concepcion-2', 'Clay'), ('gran-canaria', 'Hard'), ('forli-5', 'Clay'), ('forli-4', 'Clay'), 
    ('forli-3', 'Clay'), ('blumenau', 'Clay'), ('forli-2', 'Clay'), ('traralgon', 'Hard'), ('bendigo', 'Hard'), ('bari', 'Clay'), ('moscow', 'Clay'), ('losinj', 'Clay'), 
    ('quito', 'Clay'), ('kyiv', 'Hard'), ('warsaw', 'Clay'), ('nottingham-2', 'Grass'), ('singapore', 'Hard'), ('potchefstroom-2', 'Hard'), ('potchefstroom', 'Hard'), ('quimper-2', 'Hard'), 
    ('sardinia', 'Clay'), ('new-york', 'Hard'), ('launceston', 'Hard'), ('ann-arbor', 'Hard'), ('liuzhou', 'Hard'), ('ningbo', 'Hard'), ('columbus-3', 'Hard'), ('kaohsiung', 'Hard'), 
    ('baotou', 'Hard'), ('portoroz', 'Hard'), ('aptos', 'Hard'), ('manerbio', 'Clay'), ('binghamton', 'Hard'), ('gatineau', 'Hard'), ('winnetka', 'Hard'), ('fergana', 'Hard'), 
    ('samarkand', 'Hard'), ('an-ning', 'Clay'), ('sophia-antipolis', 'Hard'), ('da-nang', 'Hard'), ('pullach', 'Clay'), ('marburg', 'Clay'), ('poprad', 'Clay'), ('caltanissetta', 'Clay'), 
    ('mestre', 'Clay'), ('loughborough', 'Hard'), ('guadalajara', 'Hard'), ('panama-city', 'Hard'), ('qujing', 'Hard'), ('kyoto', 'Hard'), ('san-francisco', 'Hard'), ('hua-hin', 'Hard'), 
    ('toyota', 'Hard'), ('suzhou', 'Hard'), ('alphen', 'Clay'), ('cortina', 'Clay'), ('qingdao', 'Hard'), ('quanzhou', 'Hard'), ('wroclaw', 'Hard'), ('tempe', 'Hard'), 
    ('happy-valley', 'Hard'), ('mons', 'Hard'), ('mohammedia', 'Clay'), ('kenitra', 'Clay'), ('meknes', 'Clay'), ('barranquilla', 'Clay'), ('fano', 'Clay'), ('trnava', 'Clay'), 
    ('manchester', 'Grass'), ('nice', 'Clay'), ('nanjing', 'Hard'), ('raanana', 'Hard'), ('jonkoping', 'Hard'), ('manila', 'Hard'), ('mendoza', 'Clay'), ('sacramento', 'Hard'), 
    ('kuala-lumpur', 'Hard'), ('agri', 'Clay'), ('eskisehir', 'Hard'), ('vercelli', 'Clay'), ('batman', 'Clay'), ('kolkata', 'Hard'), ('traralgon-2', 'Hard'), ('reunion-island', 'Hard'), 
    ('indore', 'Hard'), ('napa', 'Hard'), ('oberstaufen', 'Clay'), ('manta', 'Hard'), ('kosice', 'Clay'), ('arad', 'Clay'), ('rimouski', 'Hard'), ('chitre', 'Hard'), 
    ('tyumen', 'Hard'), ('yeongwol', 'Hard'), ('petange', 'Hard'), ('timisoara', 'Clay'), ('kun-ming', 'Hard'), ('rio-quente', 'Hard'), ('sarajevo', 'Hard'), ('villa-allende', 'Clay'), 
    ('lermontov', 'Clay'), ('wuhan', 'Hard'), ('bercuit', 'Clay'), ('penza', 'Hard'), ('rabat', 'Clay'), ('wolfsburg', 'Hard'), ('caloundra', 'Hard'), ('palermo', 'Clay'), ('belo-horizonte', 'Clay'),
    ('tulsa', 'Hard'), ('trani', 'Clay'), ('dortmund', 'Clay'), ('rijeka', 'Hard'), ('alessandria', 'Clay'), ('courmayeur', 'Hard'), ('cancun', 'Hard'), 
    ('tarragona', 'Clay'), ('salvador', 'Hard'), ('rimini', 'Clay'), ('bytom', 'Clay'), ('rome-3', 'Clay'), ('ojai', 'Hard'), ('san-remo', 'Clay'), ('cairo', 'Clay'), 
    ('sunrise', 'Hard'), ('jersey-2', 'Hard'), ('freudenstadt', 'Clay'), ('constanta', 'Clay'), ('korat', 'Hard'), ('iquique', 'Hard'), ('poertschach', 'Clay'), ('long-island', 'Hard'), 
    ('stuttgart-1', 'Hard'), ('verizon-tennis-challenge', 'Hard'), ('grand-slam-cup', 'Hard'), ('philadelphia', 'Hard'), ('merano', 'Clay'), ('schenectady', 'Hard'), ('pinehurst', 'Clay'), 
    ('osaka', 'Hard'), ('oahu', 'Hard'), ('genoa', 'Clay'), ('wembley', 'Hard'), ('itaparica', 'Hard'), ('toronto-2', 'Hard'),
    ('miami', 'Hard'), ('morelia', 'Clay'), ('bengaluru', 'Hard'), ('montemar', 'Clay'), ('paris', 'Hard'), ('vienna', 'Hard'), ('valencia', 'Clay'), ('shanghai', 'Hard'), 
    ('fairfield', 'Hard'), ('bad-waltersdorf', 'Clay'), ('us-open', 'Hard'), ('zhangjiagang', 'Hard'), ('washington', 'Hard'), ('porto', 'Clay'), ('umag', 'Clay'), 
    ('wimbledon', 'Grass'), ('ismaning', 'Carpet'), ('santa-fe-2', 'Clay'), ('toronto', 'Hard'), ('hopman-cup', 'Hard'), ('parma', 'Clay'), ('coquimbo', 'Clay'), ('coquimbo-2', 'Clay'), 
    ('salvador-de-bahia', 'Clay'), ('bengaluru-2', 'Hard'), ('rome-2', 'Clay'), ('new-haven', 'Hard'), ('ludwigshafen', 'Clay'), ('nanchang', 'Hard'), ('karshi', 'Hard'), 
    ('le-gosier', 'Hard'), ('brescia', 'Clay'), ('izmir', 'Hard'), ('memphis', 'Hard'), ('turin', 'Hard'), ('atp-challenger-tour-finals', 'Hard'), ('cali-2', 'Clay'), 
    ('sao-jose-do-rio-preto', 'Clay'), ('orbetello', 'Clay'), ('johannesburg', 'Hard'), ('san-jose', 'Hard'), ('belem', 'Clay'), ('los-angeles', 'Hard'), ('bath', 'Hard'), 
    ('pingguo', 'Hard'), ('honolulu', 'Hard'), ('recife-2', 'Clay'), ('campos-do-jordao', 'Hard'), ('wuhai', 'Hard'), ('cremona', 'Clay'), ('recife', 'Clay'), ('sainte-clotilde', 'Hard'), 
    ('aachen', 'Clay'), ('calabasas', 'Hard'), ('saransk', 'Clay'), ('carson', 'Hard'), ('ramat-hasharon', 'Hard'), ('baton-rouge', 'Hard'), ('jersey', 'Hard'), 
    ('khanty-mansiysk', 'Hard'), ('chuncheon', 'Hard'), ('vigo', 'Clay'), ('mediterranean-games', 'Clay'), ('yuba-city', 'Hard'), ('karlsruher', 'Clay'), 
    ('carson-2', 'Hard'), ('guinot-mary-cohr', 'Hard'), ('besancon', 'Hard'), ('dnepropetrovsk', 'Hard'), ('mumbai', 'Hard'), ('shanghai-2', 'Hard'), 
    ('new-haven-2', 'Hard'), ('bologna', 'Clay'), ('kuala-lumpur-2', 'Hard'), ('zaragoza', 'Clay'), ('wellington', 'Hard'), ('guaruja', 'Clay'), ('maceio', 'Clay')

]

superficie_dict = dict(torneos_superficies)
masc['superficie'] = masc['torneo'].map(superficie_dict)

masc.loc[masc['torneo'] == 'memphis', 'ano'] = 2017
masc.loc[masc['torneo'] == 'merano', 'ano'] = 1997
masc.loc[masc['torneo'] == 'metz', 'ano'] = 2024
masc[['dia', 'hora']] = masc['fecha'].str.split(' ', expand=True) #pongo la fecha como de normal (13/01/2025)
masc['dia'] = masc['dia'].str.replace('.', '/', regex=False)
masc['ano'] = masc['ano'].apply(lambda x: str(int(x)) if pd.notna(x) else x)
masc['dia'] = masc['dia'].astype(str).str.strip()
masc['ano'] = masc['ano'].astype(str).str.strip()
masc['fecha'] = masc['dia'] + masc['ano']
masc = masc.drop(columns = ['dia', 'cuotaA', 'cuotaB'])
masc['fecha'] = pd.to_datetime(masc['fecha'], format='%d/%m/%Y')

filtro = (masc['fecha'].dt.year == 2025) & (masc['fecha'].dt.month == 12)
masc.loc[filtro, 'fecha'] = masc.loc[filtro, 'fecha'] - pd.DateOffset(years=1)

masc['sets'] = masc['setsA'] + masc['setsB']

#Me quedo con las cuotas buenas
masc = masc.rename(columns={
    'cuotaA_2': 'cuotaA',
    'cuotaB_2': 'cuotaB'
})

#Le quito el símbolo del porcentaje a las columnas que lo tienen
cols_porcentaje = [col for col in masc.columns if 'porcentaje' in col.lower()]
for col in cols_porcentaje:
    masc[col] = masc[col].astype(str).str.replace('%', '', regex=False)
    masc[col] = pd.to_numeric(masc[col], errors='coerce')

#Le quito los paréntesis a las columnas que lo tienen
cols_con_par = [col for col in masc.columns if masc[col].astype(str).str.contains(r'\(\d+/\d+\)').any()]
for col in cols_con_par:
    masc[col] = masc[col].astype(str).str.replace('(', '', regex=False).str.replace(')', '', regex=False)

columnas_a_eliminar = ['juegosGanadosPrimerSaqueA', 'juegosGanadosPrimerSaqueB', 'JuegosGanadosSegundoSaqueA', 'JuegosGanadosSegundoSaqueB', 'BreakPointsSalvadosA', 'BreakPointsSalvadosB', 'PuntosBreakConvertidosA', 'PuntosBreakConvertidosB',
    'PuntosGanadosAlRestoConPrimerSaqueA', 'PuntosGanadosAlRestoConPrimerSaqueB', 'PuntosGanadosAlRestoA', 'PuntosGanadosAlRestoB', 'PuntosGanadosAlRestoConSegundoSaqueA', 'PuntosGanadosAlRestoConSegundoSaqueB', 'PuntosGanadosAlSaqueA', 
    'PuntosGanadosAlSaqueB', 'PuntosGanadosA', 'PuntosGanadosB', 'JuegosGanadosAlSaqueA', 'JuegosGanadosAlSaqueB', 'JuegosGanadosAlRestoA', 'JuegosGanadosAlRestoB', 'JuegosGanadosA', 'JuegosGanadosB', 'PuntosGanadosRedA', 
    'PuntosGanadosRedB', 'ano']

masc = masc.drop(columns = columnas_a_eliminar)

#Creo la columna ganador

masc['ganador'] = -1

ganadorA1 = filtroGrandSlam & filtroGrandSlamGanaA
masc.loc[ganadorA1, 'ganador'] = 0

ganadorB1 = filtroGrandSlam & filtroGrandSlamGanaB
masc.loc[ganadorB1, 'ganador'] = 1

ganadorA2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaA
masc.loc[ganadorA2, 'ganador'] = 0

ganadorB2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaB
masc.loc[ganadorB2, 'ganador'] = 1

masc['retirado'] = masc['retirado'].map({'si': 1, 'no': 0}).astype(int)

masc = masc[masc['retirado'] == 0]
masc = masc.drop(columns = ['retirado'])

masc = masc[masc['acesA'] != '31.05.2025 00:16']
masc = masc.sort_values('fecha', ascending=False).reset_index(drop=True)

pd.set_option('display.max_columns', None)
masc.head()


,torneo,jugadorA,jugadorB,setsA,setsB,fecha,acesA,acesB,dobleFaltasA,dobleFaltasB,porcentajePrimerSaqueA,porcentajePrimerSaqueB,porcentajePuntosGanadosConPrimerSaqueA,porcentajePuntosGanadosConPrimerSaqueB,puntosGanadosConPrimerSaqueA,puntosGanadosConPrimerSaqueB,porcentajePuntosGanadosConSegundoSaqueA,porcentajePuntosGanadosConSegundoSaqueB,puntosGanadosConSegundoSaqueA,puntosGanadosConSegundoSaqueB,porcentajeBreakPointsSalvadosA,porcentajeBreakPointsSalvadosB,breakPointsSalvadosA,breakPointsSalvadosB,porcentajeBreakPointsA,porcentajeBreakPointsB,breakPointsA,breakPointsB,velocidadMediaPrimerSaqueA,velocidadMediaPrimerSaqueB,velocidadMediaSegundoSaqueA,velocidadMediaSegundoSaqueB,porcentajePuntosGanadosAlRestoConPrimerSaqueA,porcentajePuntosGanadosAlRestoConPrimerSaqueB,puntosGanadosAlRestoConPrimerSaqueA,puntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoConSegundoSaqueA,porcentajePuntosGanadosAlRestoConSegundoSaqueB,puntosGanadosAlRestoConSegundoSaqueA,puntosGanadosAlRestoConSegundoSaqueB,winnersA,winnersB,erroresNoForzadosA,erroresNoForzadosB,porcentajePuntosGanadosRedA,porcentajePuntosGanadosRedB,puntosGanadosRedA,puntosGanadosRedB,porcentajePuntosGanadosSaqueA,porcentajePuntosGanadosSaqueB,puntosGanadosSaqueA,puntosGanadosSaqueB,porcentajePuntosGanadosRestoA,porcentajePuntosGanadosRestoB,puntosGanadosRestoA,puntosGanadosRestoB,porcentajePuntosGanadosA,porcentajePuntosGanadosB,puntosGanadosA,puntosGanadosB,puntosGanadosUltimas10BolasA,puntosGanadosUltimas10BolasB,puntosDePartidoSalvadosA,puntosDePartidoSalvadosB,porcentajeJuegosGanadosSaqueA,porcentajeJuegosGanadosSaqueB,juegosGanadosSaqueA,juegosGanadosSaqueB,porcentajeJuegosGanadosRestoA,porcentajeJuegosGanadosRestoB,juegosGanadosRestoA,juegosGanadosRestoB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,juegosGanadosA,juegosGanadosB,cuotaA,cuotaB,dobleA,dobleB,porcentajeJuegosGanadosPrimerSaqueA,porcentajeJuegosGanadosPrimerSaqueB,porcentajeJuegosGanadosSegundoSaqueA,porcentajeJuegosGanadosSegundoSaqueB,porcentajePuntosGanadosAlRestoA,porcentajePuntosGanadosAlRestoB,porcentajePuntosBreakConvertidosA,porcentajePuntosBreakConvertidosB,porcentajePuntosGanadosAlSaqueA,porcentajePuntosGanadosAlSaqueB,PuntosGanadosUltimas10BolasA,PuntosGanadosUltimas10BolasB,BolasDePartidoSalvadasA,BolasDePartidoSalvadasB,porcentajeJuegosGanadosAlSaqueA,porcentajeJuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,superficie,hora,sets,ganador
0,royan,Paris T.,De Schepper K.,2,1,2025-06-16,1,3,NaN,NaN,58.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,29.0,NaN,NaN,61.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,40.0,NaN,NaN,1.70,2.00,3.0,5.0,71.0,75.0,63.0,39.0,39.0,32.0,100.0,40.0,68.0,61.0,8.0,2.0,0.0,1.0,87.0,67.0,33.0,13.0,Clay,11:10,3,0
1,sassuolo,Bocchi L.,Nijboer R.,0,2,2025-06-16,3,2,NaN,NaN,55.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,35.0,NaN,NaN,44.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,67.0,NaN,NaN,3.80,1.20,4.0,2.0,65.0,66.0,36.0,56.0,37.0,48.0,33.0,50.0,52.0,63.0,4.0,6.0,0.0,0.0,56.0,89.0,11.0,44.0,Clay,12:35,2,1
2,royan,Alcala Gurri M.,Beauge M.,2,0,2025-06-16,1,2,NaN,NaN,79.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,22.0,NaN,NaN,59.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,25.0,NaN,NaN,1.36,2.70,2.0,2.0,78.0,53.0,60.0,41.0,52.0,26.0,44.0,0.0,74.0,48.0,8.0,2.0,0.0,0.0,100.0,50.0,50.0,0.0,Clay,12:45,2,0
3,royan,Marionneau L.,Santamarta Roig A.,0,2,2025-06-16,0,1,NaN,NaN,77.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0

Ahora vamos a tratar los nan

In [625]:
filtro = masc.isna().sum() == 0
print(len(masc.isna().sum()[filtro]))
print(masc.isna().sum()[filtro].index.tolist())

10
['torneo', 'jugadorA', 'jugadorB', 'setsA', 'setsB', 'fecha', 'superficie', 'hora', 'sets', 'ganador']


In [626]:
filtro = (masc.isna().sum() > 0) & (masc.isna().sum() <= 3000)
masc.isna().sum()[filtro]
print(masc.isna().sum()[filtro].index.tolist())

['cuotaA', 'cuotaB']


In [627]:
filtro = (masc.isna().sum() > 3000) & (masc.isna().sum() <= 10200)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
#masc = masc.drop(columns = columnas_a_eliminar)

30
['acesA', 'acesB', 'porcentajePrimerSaqueA', 'porcentajePrimerSaqueB', 'porcentajeBreakPointsSalvadosA', 'porcentajeBreakPointsSalvadosB', 'porcentajePuntosGanadosAlRestoConPrimerSaqueA', 'porcentajePuntosGanadosAlRestoConPrimerSaqueB', 'porcentajePuntosGanadosAlRestoConSegundoSaqueA', 'porcentajePuntosGanadosAlRestoConSegundoSaqueB', 'porcentajePuntosGanadosA', 'porcentajePuntosGanadosB', 'porcentajeJuegosGanadosA', 'porcentajeJuegosGanadosB', 'dobleA', 'dobleB', 'porcentajeJuegosGanadosPrimerSaqueA', 'porcentajeJuegosGanadosPrimerSaqueB', 'porcentajeJuegosGanadosSegundoSaqueA', 'porcentajeJuegosGanadosSegundoSaqueB', 'porcentajePuntosGanadosAlRestoA', 'porcentajePuntosGanadosAlRestoB', 'porcentajePuntosBreakConvertidosA', 'porcentajePuntosBreakConvertidosB', 'porcentajePuntosGanadosAlSaqueA', 'porcentajePuntosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlSaqueA', 'porcentajeJuegosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlRestoA', 'porcentajeJuegosGanadosAlRestoB']


In [628]:
filtro = (masc.isna().sum() > 10200) & (masc.isna().sum() <= 15900)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

4
['PuntosGanadosUltimas10BolasA', 'PuntosGanadosUltimas10BolasB', 'BolasDePartidoSalvadasA', 'BolasDePartidoSalvadasB']


In [629]:
filtro = (masc.isna().sum() > 15900) & (masc.isna().sum() <= 29000)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

10
['velocidadMediaPrimerSaqueA', 'velocidadMediaPrimerSaqueB', 'velocidadMediaSegundoSaqueA', 'velocidadMediaSegundoSaqueB', 'winnersA', 'winnersB', 'erroresNoForzadosA', 'erroresNoForzadosB', 'porcentajePuntosGanadosRedA', 'porcentajePuntosGanadosRedB']


In [630]:
filtro = (masc.isna().sum() > 29000)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

46
['dobleFaltasA', 'dobleFaltasB', 'porcentajePuntosGanadosConPrimerSaqueA', 'porcentajePuntosGanadosConPrimerSaqueB', 'puntosGanadosConPrimerSaqueA', 'puntosGanadosConPrimerSaqueB', 'porcentajePuntosGanadosConSegundoSaqueA', 'porcentajePuntosGanadosConSegundoSaqueB', 'puntosGanadosConSegundoSaqueA', 'puntosGanadosConSegundoSaqueB', 'breakPointsSalvadosA', 'breakPointsSalvadosB', 'porcentajeBreakPointsA', 'porcentajeBreakPointsB', 'breakPointsA', 'breakPointsB', 'puntosGanadosAlRestoConPrimerSaqueA', 'puntosGanadosAlRestoConPrimerSaqueB', 'puntosGanadosAlRestoConSegundoSaqueA', 'puntosGanadosAlRestoConSegundoSaqueB', 'puntosGanadosRedA', 'puntosGanadosRedB', 'porcentajePuntosGanadosSaqueA', 'porcentajePuntosGanadosSaqueB', 'puntosGanadosSaqueA', 'puntosGanadosSaqueB', 'porcentajePuntosGanadosRestoA', 'porcentajePuntosGanadosRestoB', 'puntosGanadosRestoA', 'puntosGanadosRestoB', 'puntosGanadosA', 'puntosGanadosB', 'puntosGanadosUltimas10BolasA', 'puntosGanadosUltimas10BolasB', 'puntosD

Vemos como se ha quedado el df

In [631]:
masc.shape

(29249, 42)

In [632]:
masc.head()

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,acesA,acesB,porcentajePrimerSaqueA,porcentajePrimerSaqueB,porcentajeBreakPointsSalvadosA,porcentajeBreakPointsSalvadosB,porcentajePuntosGanadosAlRestoConPrimerSaqueA,porcentajePuntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoConSegundoSaqueA,porcentajePuntosGanadosAlRestoConSegundoSaqueB,porcentajePuntosGanadosA,porcentajePuntosGanadosB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,cuotaA,cuotaB,dobleA,dobleB,porcentajeJuegosGanadosPrimerSaqueA,porcentajeJuegosGanadosPrimerSaqueB,porcentajeJuegosGanadosSegundoSaqueA,porcentajeJuegosGanadosSegundoSaqueB,porcentajePuntosGanadosAlRestoA,porcentajePuntosGanadosAlRestoB,porcentajePuntosBreakConvertidosA,porcentajePuntosBreakConvertidosB,porcentajePuntosGanadosAlSaqueA,porcentajePuntosGanadosAlSaqueB,porcentajeJuegosGanadosAlSaqueA,porcentajeJuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,superficie,hora,sets,ganador
0,royan,Paris T.,De Schepper K.,2,1,2025-06-16,1,3,58.0,61.0,60.0,0.0,25.0,29.0,61.0,37.0,54.0,46.0,60.0,40.0,1.70,2.00,3.0,5.0,71.0,75.0,63.0,39.0,39.0,32.0,100.0,40.0,68.0,61.0,87.0,67.0,33.0,13.0,Clay,11:10,3,0
1,sassuolo,Bocchi L.,Nijboer R.,0,2,2025-06-16,3,2,55.0,75.0,50.0,67.0,34.0,35.0,44.0,64.0,44.0,56.0,33.0,67.0,3.80,1.20,4.0,2.0,65.0,66.0,36.0,56.0,37.0,48.0,33.0,50.0,52.0,63.0,56.0,89.0,11.0,44.0,Clay,12:35,2,1
2,royan,Alcala Gurri M.,Beauge M.,2,0,2025-06-16,1,2,79.0,61.0,100.0,56.0,47.0,22.0,59.0,40.0,62.0,38.0,75.0,25.0,1.36,2.70,2.0,2.0,78.0,53.0,60.0,41.0,52.0,26.0,44.0,0.0,74.0,48.0,100.0,50.0,50.0,0.0,Clay,12:45,2,0
3,royan,Marionneau L.,Santamarta Roig A.,0,2,2025-06-16,0,1,77.0,64.0,33.0,100.0,28.0,50.0,45.0,67.0,39.0,61.0,20.0,80.0,3.95,1.20,2.0,4.0,50.0,72.0,33.0,55.0,34.0,54.0,0.0,67.0,46.0,66.0,43.0,100.0,0.0,57.0,Clay,11:20,2,1
4,royan,Poullain L.,Bouquet L.,2,0,2025-06-16,5,2,76.0,55.0,50.0,50.0,30.0,36.0,60.0,38.0,53.0,47.0,57.0,43.0,1.36,2.75,1.0,3.0,64.0,70.0,63.0,40.0,43.0,36.0,50.0,50.0,64.0,57.0,80.0,64.0,36.0,20.0,Clay,11:20,2,0


Más transformaciones

In [633]:
masc.loc[(masc['porcentajePuntosGanadosAlSaqueA'].isna()) & (masc['fecha'] > '2020-12-31'), 'fecha'].value_counts().sum() # 185 partidos a partir de 2021 que no tienen casi registros.

np.int64(185)

In [634]:
masc = masc[~masc['porcentajePuntosGanadosAlSaqueA'].isna()]
masc.shape

(19458, 42)

In [ ]:
#masc.to_csv(".\\data\\data2\\datosmasc_2025-06-18.csv", index=False, encoding='utf-8-sig')
masc.head()

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,acesA,acesB,porcentajePrimerSaqueA,porcentajePrimerSaqueB,porcentajeBreakPointsSalvadosA,porcentajeBreakPointsSalvadosB,porcentajePuntosGanadosAlRestoConPrimerSaqueA,porcentajePuntosGanadosAlRestoConPrimerSaqueB,porcentajePuntosGanadosAlRestoConSegundoSaqueA,porcentajePuntosGanadosAlRestoConSegundoSaqueB,porcentajePuntosGanadosA,porcentajePuntosGanadosB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,cuotaA,cuotaB,dobleA,dobleB,porcentajeJuegosGanadosPrimerSaqueA,porcentajeJuegosGanadosPrimerSaqueB,porcentajeJuegosGanadosSegundoSaqueA,porcentajeJuegosGanadosSegundoSaqueB,porcentajePuntosGanadosAlRestoA,porcentajePuntosGanadosAlRestoB,porcentajePuntosBreakConvertidosA,porcentajePuntosBreakConvertidosB,porcentajePuntosGanadosAlSaqueA,porcentajePuntosGanadosAlSaqueB,porcentajeJuegosGanadosAlSaqueA,porcentajeJuegosGanadosAlSaqueB,porcentajeJuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,superficie,hora,sets,ganador
0,royan,Paris T.,De Schepper K.,2,1,2025-06-16,1,3,58.0,61.0,60.0,0.0,25.0,29.0,61.0,37.0,54.0,46.0,60.0,40.0,1.70,2.00,3.0,5.0,71.0,75.0,63.0,39.0,39.0,32.0,100.0,40.0,68.0,61.0,87.0,67.0,33.0,13.0,Clay,11:10,3,0
1,sassuolo,Bocchi L.,Nijboer R.,0,2,2025-06-16,3,2,55.0,75.0,50.0,67.0,34.0,35.0,44.0,64.0,44.0,56.0,33.0,67.0,3.80,1.20,4.0,2.0,65.0,66.0,36.0,56.0,37.0,48.0,33.0,50.0,52.0,63.0,56.0,89.0,11.0,44.0,Clay,12:35,2,1
2,royan,Alcala Gurri M.,Beauge M.,2,0,2025-06-16,1,2,79.0,61.0,100.0,56.0,47.0,22.0,59.0,40.0,62.0,38.0,75.0,25.0,1.36,2.70,2.0,2.0,78.0,53.0,60.0,41.0,52.0,26.0,44.0,0.0,74.0,48.0,100.0,50.0,50.0,0.0,Clay,12:45,2,0
3,royan,Marionneau L.,Santamarta Roig A.,0,2,2025-06-16,0,1,77.0,64.0,33.0,100.0,28.0,50.0,45.0,67.0,39.0,61.0,20.0,80.0,3.95,1.20,2.0,4.0,50.0,72.0,33.0,55.0,34.0,54.0,0.0,67.0,46.0,66.0,43.0,100.0,0.0,57.0,Clay,11:20,2,1
4,royan,Poullain L.,Bouquet L.,2,0,2025-06-16,5,2,76.0,55.0,50.0,50.0,30.0,36.0,60.0,38.0,53.0,47.0,57.0,43.0,1.36,2.75,1.0,3.0,64.0,70.0,63.0,40.0,43.0,36.0,50.0,50.0,64.0,57.0,80.0,64.0,36.0,20.0,Clay,11:20,2,0
